## 1.Import Library

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import string
import re
from pyvi import ViTokenizer

## 2.Explore Data Analysis

In [3]:
df_train = pd.read_csv('./DataPhone/Train.csv')
df_test=pd.read_csv('./DataPhone/Test.csv')

In [4]:
df_train.columns

Index(['index', 'comment', 'n_star', 'date_time', 'label'], dtype='object')

In [5]:
df_train.head(5)

,index,comment,n_star,date_time,label
0,0,Mới mua máy này Tại thegioididong thốt nốt cảm...,5,2 tuần trước,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,1,Pin kém còn lại miễn chê mua 8/3/2019 tình trạ...,5,14/09/2019,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,2,Sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,3,17/08/2020,{FEATURES#Negative};
3,3,"Mọi người cập nhật phần mềm lại , nó sẽ bớt tố...",3,29/02/2020,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,4,"Mới mua Sài được 1 tháng thấy pin rất trâu, Sà...",5,4/6/2020,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


### 2.1.Overview of phone dataset

> The dataset consists of 11,122 comments, including of four features: 

>comment: Commentary content.
 n_star: The user evaluates the smartphone's star.

>data_time: The date and time the comment was posted.

>label: Label of comment.

>All samples are in text format. No tokenization has been applied. Users of this dataset are free to use whatever sentence representation they choose.

### 2.2 Structure of dataset

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7786 entries, 0 to 7785
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      7786 non-null   int64 
 1   comment    7786 non-null   object
 2   n_star     7786 non-null   int64 
 3   date_time  7786 non-null   object
 4   label      7786 non-null   object
dtypes: int64(2), object(3)
memory usage: 304.3+ KB


### 2.3 Summary of statistic

In [7]:
df_train.describe()

,index,n_star
count,7786.000000,7786.000000
mean,3892.500000,3.706011
std,2247.768931,1.508531
min,0.000000,1.000000
25%,1946.250000,3.000000
50%,3892.500000,4.000000
75%,5838.750000,5.000000
max,7785.000000,5.000000


### 2.4. Preprocessing and Visualization of the dataset

> Handle punctuation, handle whitespace, handle icons in strings

In [8]:
def remove_pucntuation(comment):
  # Create a translation table
  translator = str.maketrans('', '', string.punctuation)
  # Remove punctuation
  new_string = comment.translate(translator)
  # Remove redudant space and break sign
  new_string = re.sub('[\n ]+', ' ', new_string)
  # Remove emoji icon
  emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                           "]+", flags=re.UNICODE)
  new_string = re.sub(emoji_pattern, '', new_string)

  return new_string

>Read  stop words file 

In [9]:
def read_filestopwords():
    with open('./DataPhone/vietnamese-stopwords.txt', 'r', encoding='utf-8') as file:
        lines = file.readlines()
        words = [line.split('\n')[0] for line in lines]
    return words

> Remove stop words

In [10]:
def remove_stopword(comment):
  stop_words = read_filestopwords()
  filtered = [word for word in comment.split() if word not in stop_words]
  return ' '.join(filtered)

> Execute function

In [11]:
df_train['comment'] = df_train['comment'].apply(lambda x: x.lower())
df_train['comment'] = df_train['comment'].apply(remove_pucntuation)
df_train['comment'] = df_train['comment'].apply(remove_stopword)
df_test['comment'] = df_test['comment'].apply(lambda x: x.lower())
df_test['comment']= df_test['comment'].apply(remove_pucntuation)
df_test['comment']= df_test['comment'].apply(remove_stopword)

> Tokenizer 

In [12]:
df_train['comment'] = df_train['comment'].apply(lambda x: ViTokenizer.tokenize(x))
df_test['comment']= df_test['comment'].apply(lambda x: ViTokenizer.tokenize(x))

In [13]:
df_train['comment'][23]

'tất_cả mọi thứ ổn hết màn_hình đẹp máy chạy êm pin sạc nhanh màu mặt lưng đẹp tiết ở phần thiết_kế chi loa ngoài đặt ở bên hông thay đặt ở cạnh tay đè loa sử_dụng í lo chính bản_thân đè quá làm loa hư thêm 1 nhỏ_xíu xiu mới mua hôm giảm 500000đ nay xem tuột xuống đươc giảm hẳn 1000000đ tức ghê'

> Observing, we can see that there are 3 types of labels: neutral, positive and negative

In [14]:
df_train.head(5)

,index,comment,n_star,date_time,label
0,0,mới mua máy thegioididong thốt_nốt cảm_thấy ok...,5,2 tuần trước,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,1,pin kém còn miễn chê mua 832019 tình_trạng pin...,5,14/09/2019,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,2,sao gọi điện_thoại màn_hình chấm nhỏ nháy gần ...,3,17/08/2020,{FEATURES#Negative};
3,3,mọi người cập_nhật phần_mềm nó bớt tốn pin mìn...,3,29/02/2020,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,4,mới mua sài 1 tháng thấy pin trâu sài bao mượt...,5,4/6/2020,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


> Function count how many characteristics belong to the labels positive, negative, neutral

In [15]:
def count_positive_labels(label):
    return label.count("Positive")
def count_neural_labels(label):
    return label.count("Neutral")
def count_negative_labels(label):
    return label.count("Negative")

> Execute function

In [16]:
df_train['positive_count']=df_train['label'].apply(count_positive_labels)
df_train['neutral_count']=df_train['label'].apply(count_neural_labels)
df_train['negative_count']=df_train['label'].apply(count_negative_labels)
df_test['positive_count']=df_test['label'].apply(count_positive_labels)
df_test['neutral_count']=df_test['label'].apply(count_neural_labels)
df_test['negative_count']=df_test['label'].apply(count_negative_labels)

> My labeling idea is that if there are many features that fall into positive then assign positive, if the most features fall into negative then assign negative, if negative is equal to neutral then assign negative, if negative is equal to positve is assigned positive, the remaining cases are assigned neutral

In [17]:
def assign_label(row):
    if row['positive_count'] > row['neutral_count'] and row['positive_count'] > row['negative_count']:
        return 'Positive'
    elif row['negative_count'] >row['neutral_count'] and row['negative_count'] > row['positive_count']:
        return 'Negative'
    elif row['negative_count'] == row['neutral_count'] :
        return 'Negative'
    elif row['neutral_count']== row ['positive_count']:
        return "Positive"
    else :
        return "Neutral"

>Execute function

In [18]:
df_train['label'] = df_train.apply(assign_label, axis=1)
df_test['label'] =df_test.apply(assign_label,axis=1)

In [19]:
df_train.head(5)

,index,comment,n_star,date_time,label,positive_count,neutral_count,negative_count
0,0,mới mua máy thegioididong thốt_nốt cảm_thấy ok...,5,2 tuần trước,Positive,6,0,0
1,1,pin kém còn miễn chê mua 832019 tình_trạng pin...,5,14/09/2019,Neutral,1,0,1
2,2,sao gọi điện_thoại màn_hình chấm nhỏ nháy gần ...,3,17/08/2020,Negative,0,0,1
3,3,mọi người cập_nhật phần_mềm nó bớt tốn pin mìn...,3,29/02/2020,Neutral,0,2,1
4,4,mới mua sài 1 tháng thấy pin trâu sài bao mượt...,5,4/6/2020,Positive,2,0,1


> 

> Remove unnecessary columns after labeling

In [20]:
list_remove=['positive_count','neutral_count','negative_count']
df_train.drop(list_remove,axis=1,inplace=True)
df_test.drop(list_remove,axis=1,inplace=True)

In [21]:
df_train.columns

Index(['index', 'comment', 'n_star', 'date_time', 'label'], dtype='object')

In [22]:
df_train.shape

(7786, 5)

>observed that there is asynchronous data in the datetime column

In [23]:
df_train.head(5)

,index,comment,n_star,date_time,label
0,0,mới mua máy thegioididong thốt_nốt cảm_thấy ok...,5,2 tuần trước,Positive
1,1,pin kém còn miễn chê mua 832019 tình_trạng pin...,5,14/09/2019,Neutral
2,2,sao gọi điện_thoại màn_hình chấm nhỏ nháy gần ...,3,17/08/2020,Negative
3,3,mọi người cập_nhật phần_mềm nó bớt tốn pin mìn...,3,29/02/2020,Neutral
4,4,mới mua sài 1 tháng thấy pin trâu sài bao mượt...,5,4/6/2020,Positive


> Function Check datetime

In [24]:
def is_valid_date(date_str):
    # Biểu thức chính quy để kiểm tra định dạng ngày tháng
    date_pattern = r'\d{1,2}/\d{1,2}/\d{4}'
    return bool(re.match(date_pattern, date_str))

> Execute function

In [25]:
valid_dates = df_train[df_train['date_time'].apply(is_valid_date)]
valid_dates_test=df_test[df_test['date_time'].apply(is_valid_date)]

> Observing we see that there are 6930 valid values

In [26]:
valid_dates.shape

(6930, 5)

In [27]:
valid_dates.head(5)

,index,comment,n_star,date_time,label
1,1,pin kém còn miễn chê mua 832019 tình_trạng pin...,5,14/09/2019,Neutral
2,2,sao gọi điện_thoại màn_hình chấm nhỏ nháy gần ...,3,17/08/2020,Negative
3,3,mọi người cập_nhật phần_mềm nó bớt tốn pin mìn...,3,29/02/2020,Neutral
4,4,mới mua sài 1 tháng thấy pin trâu sài bao mượt...,5,4/6/2020,Positive
5,5,xài tốt mượt pin trâu bạn độ sáng đủ nhân_viên...,5,20/06/2019,Positive


In [28]:
invalid_dates=df_train[~df_train['date_time'].apply(is_valid_date)]
invalid_dates_test=df_test[~df_test['date_time'].apply(is_valid_date)]

> There are 856 values in the datetime column out of a total of 6930 that have inconsistent formatting

In [29]:
invalid_dates.shape

(856, 5)

In [30]:
invalid_dates.head(5)

,index,comment,n_star,date_time,label
0,0,mới mua máy thegioididong thốt_nốt cảm_thấy ok...,5,2 tuần trước,Positive
6,6,mình mới xài 7 tháng xuống 7 pin chả hiểu máy ...,1,1 tuần trước,Negative
11,11,khá ổn trong tầm giá cam đẹp sạc nhanh màn_hìn...,5,3 tuần trước,Positive
14,14,mình mua dc 1 tuần máy phát trực_tiếp tự_nhiên...,1,6 ngày trước,Negative
18,18,dùng đơ máy bấm mãi mới khởi_động song nóng ra...,1,5 ngày trước,Negative


> Fill in the common value for that attribute

In [31]:
common_value=valid_dates['date_time'].mode()[0]
common_value_test=valid_dates_test['date_time'].mode()[0]
invalid_dates.loc[:, 'date_time'] = common_value
invalid_dates_test.loc[:,'date_time']=common_value

In [32]:
invalid_dates.head(5)

,index,comment,n_star,date_time,label
0,0,mới mua máy thegioididong thốt_nốt cảm_thấy ok...,5,21/04/2020,Positive
6,6,mình mới xài 7 tháng xuống 7 pin chả hiểu máy ...,1,21/04/2020,Negative
11,11,khá ổn trong tầm giá cam đẹp sạc nhanh màn_hìn...,5,21/04/2020,Positive
14,14,mình mua dc 1 tuần máy phát trực_tiếp tự_nhiên...,1,21/04/2020,Negative
18,18,dùng đơ máy bấm mãi mới khởi_động song nóng ra...,1,21/04/2020,Negative


> combine invalid_dates and valid_dates

In [33]:
df_train=pd.concat([valid_dates,invalid_dates])
df_test=pd.concat ([valid_dates_test,invalid_dates_test])

In [34]:
df_train.shape

(7786, 5)

In [35]:
df_train.head(5)

,index,comment,n_star,date_time,label
1,1,pin kém còn miễn chê mua 832019 tình_trạng pin...,5,14/09/2019,Neutral
2,2,sao gọi điện_thoại màn_hình chấm nhỏ nháy gần ...,3,17/08/2020,Negative
3,3,mọi người cập_nhật phần_mềm nó bớt tốn pin mìn...,3,29/02/2020,Neutral
4,4,mới mua sài 1 tháng thấy pin trâu sài bao mượt...,5,4/6/2020,Positive
5,5,xài tốt mượt pin trâu bạn độ sáng đủ nhân_viên...,5,20/06/2019,Positive


>Remove index column

In [36]:
df_train.drop("index",axis=1,inplace=True)
df_test.drop("index",axis=1,inplace=True)

In [37]:
df_train.shape

(7786, 4)

In [38]:
df_train.head(5)

,comment,n_star,date_time,label
1,pin kém còn miễn chê mua 832019 tình_trạng pin...,5,14/09/2019,Neutral
2,sao gọi điện_thoại màn_hình chấm nhỏ nháy gần ...,3,17/08/2020,Negative
3,mọi người cập_nhật phần_mềm nó bớt tốn pin mìn...,3,29/02/2020,Neutral
4,mới mua sài 1 tháng thấy pin trâu sài bao mượt...,5,4/6/2020,Positive
5,xài tốt mượt pin trâu bạn độ sáng đủ nhân_viên...,5,20/06/2019,Positive


In [39]:
df_train['comment'][0]

'mới mua máy thegioididong thốt_nốt cảm_thấy ok bin trâu chụp ảnh đẹp loa nghe to bắt wf khỏe_sóng ổn_định giá_thành túi_tiền nhân_viên tư_vấn nhiệt_tình'

> Dump file CSV after data processing

In [40]:
df_train.to_csv("./DataPhone/trainprocessed.csv", index=False)
df_test.to_csv("./DataPhone/testprocesssed.csv",index=False)